In [ ]:
import pandas as pd
import numpy as np
import pygmt
from collections import Counter
import os
from scipy.spatial.distance import cdist

In [ ]:
df_road = pd.read_csv("/home/velgueta/notebooks/project_Mt-Rainier_DAS/Notebooks/Paradise2NisquallyEntrace_Channels.csv")
df_road.head()

In [ ]:
# list of seismic? stations in Mt Rainier
df_seis = pd.read_csv("/home/velgueta/notebooks/project_Mt-Rainier_DAS/Text-files/MtRainier_seis.txt",sep="|")
#print(df_seis.head())
#df_seis["Station"]

In [ ]:
# create short station list including stations which detect avalanche on 2023-12-10T12:09
sta_list = ['RER', 'RCS', 'RCM', 'LON', 'LO2', 'SIFT', 'RUSH', 'PR05', 'PR04', 'PARA', 'PANH', 'OBSR', 'ARAT']
df_seis_short = df_seis[df_seis["Station"].isin(sta_list)]
df_seis_short

In [ ]:
# loading swarm 

df_swarm = pd.read_csv("/home/velgueta/notebooks/project_Mt-Rainier_DAS/Text-files/pnsn_event_export_5dias.csv")


In [ ]:
df_seismicity = pd.read_csv('/home/velgueta/notebooks/project_Mt-Rainier_DAS/Text-files/pnsn_event_export_20240613.csv')

In [ ]:
df_template = pd.read_csv("/home/velgueta/notebooks/project_Mt-Rainier_DAS/Text-files/Template-location.csv")
df_template.head()

In [ ]:
df_seismicity["Depth Km"].max()

In [ ]:
import numpy as np

# Define region
RR = [-122.1, -121.45, 46.65, 47.]
grid = pygmt.datasets.load_earth_relief(resolution="03s", region=RR)

# Extraer valores mínimo y máximo de elevación correctamente
grid_min = float(grid.min().values)
grid_max = float(grid.max().values)

# Crear hillshade
shading = pygmt.grdgradient(grid, azimuth=315)

# Crear un colormap fijo para la elevación (para que no cambie en diferentes mapas)
pygmt.makecpt(cmap="geo", series=[grid_min, grid_max], continuous=True)

# Crear la figura
fig = pygmt.Figure()

with pygmt.config(FONT_ANNOT_PRIMARY="15p", MAP_FRAME_TYPE="plain"):
    fig.grdimage(grid=grid, shading=shading, projection="M15c", frame="a", 
                 cmap="geo", transparency=30)  # Fijar colormap de elevación

# Agregar barra de color para la elevación
with pygmt.config(FONT_LABEL="20p"):
    fig.colorbar(position='JMB+w15c/0.5c+o0c/1c', frame='af+l"Elevation (m)"')


# Plot del camino para DAS
fig.plot(x=df_road['Longitude [°]'], y=df_road['Latitude [°]'], pen='3p,purple', label='DAS array')

# Plot de estaciones sísmicas
fig.plot(x=df_seis["Longitude"], y=df_seis["Latitude"],
         style="i0.50c", pen="1p,black", fill="black", transparency=30)

# Crear mapa de colores para profundidad basado en Scientific Colour Maps
pygmt.makecpt(cmap="roma.cpt", 
              series=[df_seismicity["Depth Km"].min(), df_seismicity["Depth Km"].max()], 
              continuous=True)

# Plot de eventos sísmicos con color por profundidad y tamaño por magnitud
fig.plot(x=df_seismicity["Lon"], y=df_seismicity["Lat"],
         style="c", pen="1p,black", fill=df_seismicity["Depth Km"], 
         cmap=True, size=0.2 + df_seismicity["Magnitude"] * 0.1)

fig.plot(x=df_swarm["Lon"], y=df_swarm["Lat"],
         style="c", pen="1p,black", fill=df_swarm["Depth Km"], 
         cmap=True, size=0.2 + df_swarm["Magnitude"] * 0.1)

fig.plot(x=df_template["Lon"], y=df_template["Lat"],
         style="c", pen="1p,black", fill=df_template["Depth Km"], 
         cmap=True, size=0.2 + df_template["Magnitude"] * 0.1)

# Crear archivo temporal para la leyenda de magnitud
legend_file = "legend.txt"
with open(legend_file, "w") as f:
    f.write("S 0.1c c 0.4c white 1p,black 0.4c M1\n")
    f.write("S 0.1c c 0.5c white 1p,black 0.5c M2\n")
    f.write("S 0.1c c 0.6c white 1p,black 0.6c M3\n")

fig.text(x=df_road['Longitude [°]'].iloc[0], 
         y=df_road['Latitude [°]'].iloc[0], 
         text="PARADISE", 
         font="14p,black", 
         justify="LT", 
         offset="0.2c/0.2c")


fig.text(x=df_road['Longitude [°]'].iloc[2000], 
         y=df_road['Latitude [°]'].iloc[2000], 
         text="LONGMIRE", 
         font="14p,black", 
         justify="LT", 
         offset="0.2c/0.2c")

# Leyenda de estaciones sísmicas
fig.plot(x=np.nan, y=np.nan, style="i0.5c", pen="0.5p,black", fill="black", 
         label='PNSN+CVO network--> Seismometer')
with pygmt.config(FONT_ANNOT="12p"):
    fig.legend(position="jTR+o0.5c/0.5c", box="+gwhite+p1p+pad=0.5c")
#fig.legend(position="jTR+o0.5c/0.5c", box="+gwhite+p1p")

with pygmt.config(FONT_ANNOT_PRIMARY="15p"):
    fig.legend(transparency=15, position="jTR+o0.2c")

# Agregar barra de color para profundidad con SCM
with pygmt.config(FONT_LABEL="20p"):
    fig.colorbar(position="JMR+w12c/0.5c+o1c/0c", frame='af+l"Depth (km)"')

# Agregar leyenda de magnitud
fig.legend(spec=legend_file, position="jBL+o0.5c/0.5c", box="+gwhite+p1p")


# Mostrar y guardar figura
fig.show()
fig.savefig("./seismicitymap.png", dpi=300)

# Eliminar archivo temporal
os.remove(legend_file)
